### 预处理

In [1]:
# import some packages
import pandas as pd
from collections import Counter
from tflearn.data_utils import pad_sequences
import random
import numpy as np
import h5py
import pickle
print("import package successful...")

import package successful...


In [2]:
# read source file as csv
base_path='./ieee_zhihu_data_old/'
train_data_x=pd.read_csv(base_path+'question_train_set3.txt',sep='\t', encoding="utf-8")
train_data_y=pd.read_csv(base_path+'question_topic_train_set3.txt',sep='\t', encoding="utf-8")
valid_data_x=pd.read_csv(base_path+'question_eval_set3.txt', sep='\t',encoding="utf-8")

train_data_x=train_data_x.fillna('')
train_data_y=train_data_y.fillna('')
valid_data_x=valid_data_x.fillna('')
print("train_data_x:",train_data_x.shape)
print("train_data_y:",train_data_y.shape)
print("valid_data_x:",valid_data_x.shape)

train_data_x: (2999967, 5)
train_data_y: (2999967, 2)
valid_data_x: (217360, 5)


In [3]:
train_data_x.head()

,question_id,title_char,title_word,desc_char,desc_word
0,6555699376639805223,"c324,c39,c40,c155,c180,c180,c181,c17,c4,c1153,...","w305,w13549,w22752,w11,w7225,w2565,w1106,w16,w...","c335,c101,c611,c189,c97,c144,c147,c101,c15,c76...","w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1..."
1,2887834264226772863,"c44,c110,c101,c286,c106,c150,c101,c892,c632,c1...","w377,w54,w285,w57,w349,w54,w108215,w6,w47986,w...","c1265,c518,c74,c131,c274,c57,c768,c769,c368,c3...","w12508,w1380,w72,w27045,w276,w111"
2,-2687466858632038806,"c15,c768,c769,c1363,c650,c1218,c2361,c11,c90,c...","w875,w15450,w42394,w15863,w6,w95421,w25,w803,w...","c693,c100,c279,c99,c189,c532,c101,c189,c145,c1...","w140340,w54,w48398,w54,w140341,w54,w12856,w54,..."
3,-5698296155734268,"c473,c1528,c528,c428,c295,c15,c101,c188,c146,c...","w8646,w2744,w1462,w9,w54,w138,w54,w50,w110,w14...",,
4,-6719100304248915192,"c190,c147,c105,c219,c220,c101,c647,c219,c220,c...","w380,w54,w674,w133,w54,w134,w614,w54,w929,w307...","c644,c1212,c253,c199,c431,c452,c424,c207,c2,c1...","w4821,w1301,w16003,w928,w1961,w2565,w50803,w11..."


In [4]:
# compute average length of title_char, title_word, desc_char, desc_word

dict_length_columns={'title_char':0,'title_word':0,'desc_char':0,'desc_word':0}
num_examples=len(train_data_x)
train_data_x_small=train_data_x.sample(frac=0.01)
for index, row in train_data_x_small.iterrows():
    title_char_length=len(row['title_char'].split(","))
    title_word_length=len(row['title_word'].split(","))
    desc_char_length=len(row['desc_char'].split(","))
    desc_word_length=len(row['desc_word'].split(","))
    dict_length_columns['title_char']=dict_length_columns['title_char']+title_char_length
    dict_length_columns['title_word']=dict_length_columns['title_word']+title_word_length
    dict_length_columns['desc_char']=dict_length_columns['desc_char']+desc_char_length
    dict_length_columns['desc_word']=dict_length_columns['desc_word']+desc_word_length
dict_length_columns={k:float(v)/float(num_examples*0.01) for k,v in dict_length_columns.items()}
print("dict_length_columns:",dict_length_columns)

# averaged length of a input. total length of all information(words,character of title+desc): 210.
# word of title+desc: 71
# character of title+desc: 139

dict_length_columns: {'desc_word': 58.48651001827687, 'title_word': 12.939208997965643, 'title_char': 22.367479375606464, 'desc_char': 117.47425888351438}


In [5]:
train_data_y.head()

,question_id,topic_ids
0,6555699376639805223,"7739004195693774975,3738968195649774859"
1,2887834264226772863,-3149765934180654494
2,-2687466858632038806,-760432988437306018
3,-5698296155734268,"-6758942141122113907,3195914392210930723"
4,-6719100304248915192,"3804601920633030746,4797226510592237555,435133..."


In [6]:
# average labels for a input
train_data_y_small=train_data_y.sample(frac=0.01)
num_examples=len(train_data_y_small)
num_labels=0
for index, row in train_data_y_small.iterrows():
    topic_ids=row['topic_ids']
    topic_id_list=topic_ids.split(",")
    num_labels+=len(topic_id_list)
average_num_labels=float(num_labels)/float(num_examples)
print("average_num_labels:",average_num_labels)


average_num_labels: 2.3392333333333335


In [7]:
word_embedding_object=open(base_path+'unused_current/word_embedding.txt')
lines_wv=word_embedding_object.readlines()
word_embedding_object.close()

In [8]:
len(lines_wv)

411721

In [11]:
lines_wv[2].split(' ')

['w11',
 '0.038359',
 '-0.006883',
 '-0.011003',
 '-0.030735',
 '0.075039',
 '-0.008816',
 '-0.062912',
 '-0.050477',
 '0.039045',
 '-0.007960',
 '0.002075',
 '-0.100198',
 '-0.022684',
 '-0.051706',
 '0.048538',
 '0.005269',
 '0.022579',
 '-0.044190',
 '-0.045492',
 '0.049457',
 '0.002136',
 '-0.058805',
 '0.009149',
 '0.053088',
 '-0.053066',
 '-0.049488',
 '0.037449',
 '0.044277',
 '0.042445',
 '-0.023130',
 '-0.030323',
 '-0.030082',
 '-0.004162',
 '-0.012156',
 '0.044964',
 '-0.047791',
 '-0.029609',
 '0.049882',
 '-0.023178',
 '-0.041648',
 '0.048316',
 '-0.028503',
 '-0.075800',
 '0.038916',
 '-0.011330',
 '-0.036790',
 '0.036794',
 '0.010653',
 '-0.025704',
 '0.060130',
 '0.014937',
 '0.106082',
 '0.055918',
 '0.000638',
 '-0.071661',
 '-0.054533',
 '0.073681',
 '-0.032574',
 '0.049205',
 '-0.018408',
 '-0.008513',
 '-0.024674',
 '-0.049137',
 '0.127848',
 '-0.027407',
 '0.041941',
 '0.076204',
 '0.017662',
 '-0.005350',
 '0.075833',
 '0.121403',
 '0.024335',
 '-0.016218',
 '0.

In [20]:
#创建word2index表和embedding表
word_embedding_object=open(base_path+'unused_current/word_embedding.txt')
lines_wv=word_embedding_object.readlines()
word_embedding_object.close()
word_list=[]
word_list.extend(['PAD','UNK','SEP'])
PAD_ID=0
UNK_ID=1
EMBED_SIZE = 200
bound = 0.5
embedding = [None] * (int(lines_wv[0].split(' ')[0]) + 3) #长度
embedding[0] = np.zeros(200)
embedding[1] = np.random.rand(200)
embedding[2] = embedding[0] + 0.1
for i, line in enumerate(lines_wv):
    if i==0: continue
    word_embedding_list=line.split(" ")
    word_token=word_embedding_list[0]
    word_list.append(word_token)
    embedding[i+2] = word_embedding_list[1:1+EMBED_SIZE]
embedding = np.array(embedding,dtype=np.float32)
# write to vocab.txt under data/ieee_zhihu_cup2
vocab_path=base_path+'vocab.txt'

vocab_word_object=open(vocab_path,'w')

word2index={}
for i, word in enumerate(word_list):
    if i<10:print(i,word)
    word2index[word]=i
    vocab_word_object.write(char+"\n")
vocab_word_object.close()
print("vocabulary of word generated....")

0 PAD
1 UNK
2 SEP
3 </s>
4 w11
5 w54
6 w6
7 w111
8 w23
9 w2828
vocabulary of word generated....


In [21]:
 # generate labels list, and save to file system 
c_labels=Counter()
train_data_y_small=train_data_y[0:100000]#.sample(frac=0.1)
for index, row in train_data_y_small.iterrows():
    topic_ids=row['topic_ids']
    topic_list=topic_ids.split(',')
    c_labels.update(topic_list)

label_list=c_labels.most_common()
label2index={}
label_target_object=open(base_path+'label_set.txt','w')
for i, label_freq in enumerate(label_list):
    label,freq=label_freq
    label2index[label]=i
    label_target_object.write(label+"\n")
    if i<20: print(label,freq)
label_target_object.close()
print("generate label dict successful...")

7476760589625268543 2308
4697014490911193675 1746
-4653836020042332281 1579
-8175048003539471998 1475
-8377411942628634656 1382
-7046289575185911002 1338
-5932391056759866388 1283
2787171473654490487 1145
-7129272008741138808 1085
-4931965624608608932 1079
2587540952280802350 1079
-6748914495015758455 1049
-5513826101327857645 993
2347973810368732059 970
9069451131871918127 958
-8132909213241034354 904
-3517637179126242000 867
-5872443091340192918 834
-3522198575349379632 830
1127459907694805235 829
generate label dict successful...


In [22]:
def transform_multilabel_as_multihot(label_list,label_size):
    """
    convert to multi-hot style
    :param label_list: e.g.[0,1,4], here 4 means in the 4th position it is true value(as indicate by'1')
    :param label_size: e.g.199
    :return:e.g.[1,1,0,1,0,0,........]
    """
    result=np.zeros(label_size)
    #set those location as 1, all else place as 0.
    result[label_list] = 1
    return result

label_list=[0,1,2,10]
label_size=20
label_list_sparse=transform_multilabel_as_multihot(label_list,label_size)
print("label_list_sparse:",label_list_sparse)

label_list_sparse: [1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [27]:
vocabulary_word2index = word2index
def get_X_Y(train_data_x,train_data_y,label_size, test_mode=False):
    """
    get X and Y given input and labels
    input:
    train_data_x:
    train_data_y:
    label_size: number of total unique labels(e.g. 1999 in this task)
    output:
    X,Y
    """
    X=[]
    Y=[]
    if test_mode:
        train_data_x_tiny_test=train_data_x[0:1000] # todo todo todo todo todo todo todo todo todo todo todo todo 
        train_data_y_tiny_test=train_data_y[0:1000] # todo todo todo todo todo todo todo todo todo todo todo todo 
    else:
        train_data_x_tiny_test=train_data_x
        train_data_y_tiny_test=train_data_y

    for index, row in train_data_x_tiny_test.iterrows():
        if index==0: 
            print(row)
            continue
        # get word of title and dssc
        title_word=row['title_word']
        desc_word=row['desc_word']
        # split into list
        title_word_list=title_word.split(',')
        desc_word_list=desc_word.split(",")
        # transform to indices
        title_word_id_list=[vocabulary_word2index.get(x,UNK_ID) for x in title_word_list if x.strip()]
        desc_word_id_list=[vocabulary_word2index.get(x,UNK_ID) for x in desc_word_list if x.strip()]
        # merge title and desc: in the middle is special token 'SEP'
        title_word_id_list.append(vocabulary_word2index['SEP'])
        title_word_id_list.extend(desc_word_id_list)
        X.append(title_word_id_list)
        if index<3: print(index,title_word_id_list)
        if index%100000==0: print(index,title_word_id_list)
    '''
    #产生单一标签
    for index, row in train_data_y_tiny_test.iterrows():
        if index==0: continue
        topic_id = row['topic_ids'].split(',')[0]
        Y.append(label2index[topic_id])
        if index%100000==0: print(index,";label_index:",label2index[topic_id])
    '''
    for index, row in train_data_y_tiny_test.iterrows():
        if index==0: continue
        topic_ids=row['topic_ids']
        topic_id_list=topic_ids.split(",")
        label_list_dense=[label2index[l] for l in topic_id_list if l.strip()]
        label_list_sparse=transform_multilabel_as_multihot(label_list_dense,label_size)
        Y.append(label_list_sparse)
        if index%100000==0: print(index,";label_list_dense:",label_list_dense)
    return X,Y

In [28]:
def save_data(cache_file_h5py,cache_file_pickle,word2index,label2index,train_X,train_Y,vaild_X,valid_Y,test_X,test_Y, embedding):
    # train/valid/test data using h5py
    f = h5py.File(cache_file_h5py, 'w')
    f['train_X'] = train_X
    f['train_Y'] = train_Y
    f['vaild_X'] = vaild_X
    f['valid_Y'] = valid_Y
    f['test_X'] = test_X
    f['test_Y'] = test_Y
    f['embedding'] = embedding
    f.close()
    # save word2index, label2index
    with open(cache_file_pickle, 'ab') as target_file:
        pickle.dump((word2index,label2index), target_file)

In [29]:
# generate training/validation/test data using source file and vocabulary/label set.
#  get X,Y---> shuffle and split data----> save to file system.
test_mode=False
label_size=len(label2index)
cache_path_h5py=base_path+'data1.h5'
cache_path_pickle=base_path+'vocab_label.pik'
max_sentence_length=100

# step 1: get (X,y) 
X,Y=get_X_Y(train_data_x,train_data_y,label_size,test_mode=test_mode)

# pad and truncate to a max_sequence_length
X = pad_sequences(X, maxlen=max_sentence_length, value=0.)  # padding to max length

# step 2. shuffle, split,
xy=list(zip(X,Y))
random.Random(10000).shuffle(xy)
X,Y=zip(*xy)
X=np.array(X); Y=np.array(Y)
num_examples=len(X)
num_valid=20000
num_valid=20000
num_train=num_examples-(num_valid+num_valid)
train_X, train_Y=X[0:num_train], Y[0:num_train]
vaild_X, valid_Y=X[num_train:num_train+num_valid], Y[num_train:num_train+num_valid]
test_X, test_Y=X[num_train+num_valid:], Y[num_train+num_valid:]
print("num_examples:",num_examples,";X.shape:",X.shape,";Y.shape:",Y.shape)
print("train_X:",train_X.shape,";train_Y:",train_Y.shape,";vaild_X.shape:",vaild_X.shape,";valid_Y:",valid_Y.shape,";test_X:",test_X.shape,";test_Y:",test_Y.shape)

# step 3: save to file system
#内存不足
save_data(cache_path_h5py,cache_path_pickle,word2index,label2index,train_X,train_Y,vaild_X,valid_Y,test_X,test_Y, embedding)
print("save cache files to file system successfully!")

del X,Y,train_X, train_Y,vaild_X, valid_Y,test_X, test_Y, embedding


question_id                                  6555699376639805223
title_char     c324,c39,c40,c155,c180,c180,c181,c17,c4,c1153,...
title_word     w305,w13549,w22752,w11,w7225,w2565,w1106,w16,w...
desc_char      c335,c101,c611,c189,c97,c144,c147,c101,c15,c76...
desc_word      w231,w54,w1681,w54,w11506,w5714,w7,w54,w744,w1...
Name: 0, dtype: object
1 [276, 5, 159, 15, 206, 5, 44345, 6, 29738, 77, 58, 1569, 15657, 7904, 7, 2, 1943, 64, 12, 3190, 329, 7]
2 [77, 5346, 3060, 2680, 6, 19537, 10, 2073, 92, 6, 3335, 96, 91, 7, 2, 328305, 5, 88392, 5, 279675, 5, 7148, 5, 313, 5, 74521, 5, 2215, 5, 181823, 5, 7815, 5, 846, 5, 9826, 5, 1238, 5, 74521, 5, 1030, 3576, 76]
100000 [402, 7508, 6, 368, 293, 10459, 7, 2, 269, 97, 23098, 125, 4, 7527, 210, 403, 246, 7508, 4, 156, 48, 2732, 402, 7508, 4840, 293, 59, 7]
200000 [664, 265, 219, 55, 190, 29649, 902, 7018, 7, 2, 6204, 1179, 219, 5, 55, 206, 9644, 190, 29649, 806, 166, 6881, 6, 219, 7]
300000 [1006, 18703, 462, 3751, 13, 28, 2122, 41, 7, 2, 45, 4

700000 ;label_list_dense: [763, 1414, 376, 197, 830]
800000 ;label_list_dense: [412, 201, 30, 4, 162]
900000 ;label_list_dense: [884]
1000000 ;label_list_dense: [271, 449, 40, 556]
1100000 ;label_list_dense: [1189]
1200000 ;label_list_dense: [1260]
1300000 ;label_list_dense: [558, 4, 12, 2]
1400000 ;label_list_dense: [524, 928, 12]
1500000 ;label_list_dense: [1173]
1600000 ;label_list_dense: [319, 71, 0, 7, 70]
1700000 ;label_list_dense: [513, 390, 269]
1800000 ;label_list_dense: [1470]
1900000 ;label_list_dense: [859, 316, 71, 242, 70]
2000000 ;label_list_dense: [645, 1839]
2100000 ;label_list_dense: [521]
2200000 ;label_list_dense: [681, 30, 381]
2300000 ;label_list_dense: [456, 1899]
2400000 ;label_list_dense: [1395, 2, 3, 35]
2500000 ;label_list_dense: [79, 54]
2600000 ;label_list_dense: [799, 1555, 670]
2700000 ;label_list_dense: [1713]
2800000 ;label_list_dense: [1561, 51]
2900000 ;label_list_dense: [509, 29, 248, 71, 128]
num_examples: 2999966 ;X.shape: (2999966, 100) ;Y.shape: 

OSError: Can't prepare for writing data (file write failed: time = Mon Feb 25 20:08:18 2019
, filename = './ieee_zhihu_data_old/data.h5', file descriptor = 58, errno = 28, error message = 'No space left on device', buf = 0x7f0e5761b530, total write size = 16760974064, bytes this sub-write = 16760974064, bytes actually written = 18446744073709551615, offset = 31758790656)

In [33]:
del X,Y

In [34]:
train_X.shape

(2959966, 100)

In [35]:
train_Y.shape

(2959966, 1999)